In [1]:
# IMPORT LIBRARY
from __future__ import print_function
import cfbd
from cfbd.rest import ApiException
import pandas as pd
from pathlib import Path
import pandas as pd
import nfl_data_py as nfl
import os
from dotenv import load_dotenv

In [2]:
# DEFINE NOTEBOOK PARAMETERS

# Load dotenv
load_dotenv()

# Set CFBD API Key
API_KEY = os.getenv('API_KEY')

# Configure CFBD API key authorization
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = API_KEY
configuration.api_key_prefix['Authorization'] = 'Bearer'

# Set export file paths
cfb_file_path = Path('../../Data/CFB/2004_to_2023_CFB_player_data.csv')
draft_file_path = Path('../../Data/Draft/2004_to_2023_draft_data.csv')
nfl_file_path = Path('../../Data/NFL/2004_to_2023_NFL_player_data.csv')

# Create Initial Dataframes
CFB_column_names = ['player', 'player_id', 'team', 'conference', 'season', 'category', 'stat_type', 'stat', 'year']
df_CFB = pd.DataFrame(columns= CFB_column_names)
draft_column_names = [
    'college_athlete_id', 'college_conference', 'college_id', 'college_team', 'height', 'city', 'country', 'country_fips',
    'latitude', 'longitude', 'state', 'name', 'nfl_athlete_id', 'nfl_team', 'overall', 'pick', 'position', 
    'pre_draft_grade', 'pre_draft_position_ranking', 'pre_draft_ranking', 'round', 'weight', 'year'
]
df_draft = pd.DataFrame(columns= draft_column_names)

# Set Dataframe Formatting Style
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.precision', 2)

In [17]:
# FUNCTION LIBRARY

def get_stats(year):
    """
    Collects the yearly CFB player data.
    
    Parameters:
    - year: int, the collection year.
    """
    # Create an instance of the API class
    api_instance = cfbd.PlayersApi(cfbd.ApiClient(configuration))
    stat_year = year

    # Attempt to collect data
    try:
        api_response = api_instance.get_player_season_stats(stat_year)
        return api_response
    except ApiException as e:
        print("Exception when calling PlayersApi->get_player_season_stats: %s\n" % e)

def get_drafts(year):
    """
    Collects the yearly draft data.
    
    Parameters:
    - year: int, the collection year.
    """

    # Create an instance of the API class
    api_instance = cfbd.DraftApi(cfbd.ApiClient(configuration))
    draft_year = year

    # Attempt to collect data
    try:
        # List of NFL Draft picks
        api_response = api_instance.get_draft_picks(year= draft_year)
        return api_response
    except ApiException as e:
        print("Exception when calling DraftApi->get_draft_picks: %s\n" % e)

def make_stats_dataframe(response):
    """
    Converts API response for CFB data into pandas dataframe.
    
    Parameters:
    - response: api_instance, the instance returned from the CFBD API.
    """

    # Intialize data list
    data_list = []

    # Loop through API response to append to data list
    for obj in response:
        # Create dictionary of data types
        data_dict = {
            'player': obj.player,  
            'player_id': obj.player_id,
            'team': obj.team,
            'conference': obj.conference,
            'season': obj.season,
            'category' : obj.category,
            'stat_type': obj.stat_type,
            'stat': obj.stat,
        }
        data_list.append(data_dict)

    # Return list as dataframe    
    df = pd.DataFrame(data_list)
    return df

def make_draft_dataframe(response):
    """
    Converts API response for draft data into pandas dataframe.
    
    Parameters:
    - response: api_instance, the instance returned from the CFBD API.
    """

    # Initialize data list
    data_list = []

    # Loop through API response to append to data list
    for obj in response:
        # Create dictionary of data types for hometown information
        hometown_info = {
            'city': obj.hometown_info.city if obj.hometown_info else None,
            'country': obj.hometown_info.country if obj.hometown_info else None,
            'country_fips': obj.hometown_info.country_fips if obj.hometown_info else None,
            'latitude': obj.hometown_info.latitude if obj.hometown_info else None,
            'longitude': obj.hometown_info.longitude if obj.hometown_info else None,
            'state': obj.hometown_info.state if obj.hometown_info else None
        }

        # Creating the flat dictionary for each draft pick
        data_dict = {
            'college_athlete_id': str(obj.college_athlete_id),
            'college_conference': obj.college_conference,
            'college_id': obj.college_id,
            'college_team': obj.college_team,
            'height': obj.height,
            'city': hometown_info['city'],
            'country': hometown_info['country'],
            'country_fips': hometown_info['country_fips'],
            'latitude': hometown_info['latitude'],
            'longitude': hometown_info['longitude'],
            'state': hometown_info['state'],
            'name': obj.name,
            'nfl_athlete_id': obj.nfl_athlete_id,
            'nfl_team': obj.nfl_team,
            'overall': obj.overall,
            'pick': obj.pick,
            'position': obj.position,
            'pre_draft_grade': obj.pre_draft_grade,
            'pre_draft_position_ranking': obj.pre_draft_position_ranking,
            'pre_draft_ranking': obj.pre_draft_ranking,
            'round': obj.round,
            'weight': obj.weight,
            'year': obj.year
        }
        data_list.append(data_dict)

    # Return list as dataframe
    df = pd.DataFrame(data_list)
    return df

In [11]:
# COLLECT CFB DATA

# Loop by year to collect the CFB data
for year in range(2004, 2024, 1):
    print(f'Retrieving CFB data for the year {year}')
    collected_df = make_stats_dataframe(get_stats(year))
    print(f'{year} CFB data has been retrieved')
    collected_df['year'] = year
    print(f'{year} has been added to the year column')
    df_CFB = pd.concat([df_CFB, collected_df], axis= 0, ignore_index= True)
    print(f'Data for the year {year} has been appended')
    print('')

# Export the dataframe as a CSV file
df_CFB.to_csv(cfb_file_path)
print('csv has been uploaded')
print('')

Retrieving CFB data for the year 2004
2004 CFB data has been retrieved
2004 has been added to the year column
Data for the year 2004 has been appended

Retrieving CFB data for the year 2005
2005 CFB data has been retrieved
2005 has been added to the year column
Data for the year 2005 has been appended

Retrieving CFB data for the year 2006
2006 CFB data has been retrieved
2006 has been added to the year column
Data for the year 2006 has been appended

Retrieving CFB data for the year 2007
2007 CFB data has been retrieved
2007 has been added to the year column
Data for the year 2007 has been appended

Retrieving CFB data for the year 2008
2008 CFB data has been retrieved
2008 has been added to the year column
Data for the year 2008 has been appended

Retrieving CFB data for the year 2009
2009 CFB data has been retrieved
2009 has been added to the year column
Data for the year 2009 has been appended

Retrieving CFB data for the year 2010
2010 CFB data has been retrieved
2010 has been add

In [3]:
# COLLECT DRAFT DATA

# Loop by year to collect the draft data
for year in range(2004, 2024, 1):
    print(f'Retrieving draft data for the year {year}')
    collected_df = make_draft_dataframe(get_drafts(year))
    print(f'{year} draft data has been retrieved')
    df_draft = pd.concat([df_draft, collected_df], axis= 0, ignore_index= True)
    print(f'Data for the year {year} has been appended')
    print('')

# Export the dataframe as a CSV file
df_draft.to_csv(draft_file_path)
print('csv has been uploaded')
print('')

Retrieving draft data for the year 2004
2004 draft data has been retrieved
Data for the year 2004 has been appended

Retrieving draft data for the year 2005


/var/folders/6c/lknfcl0j3lb3skfwn995xkym0000gn/T/ipykernel_11096/2499207334.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_draft = pd.concat([df_draft, collected_df], axis= 0, ignore_index= True)


2005 draft data has been retrieved
Data for the year 2005 has been appended

Retrieving draft data for the year 2006
2006 draft data has been retrieved
Data for the year 2006 has been appended

Retrieving draft data for the year 2007
2007 draft data has been retrieved
Data for the year 2007 has been appended

Retrieving draft data for the year 2008
2008 draft data has been retrieved
Data for the year 2008 has been appended

Retrieving draft data for the year 2009
2009 draft data has been retrieved
Data for the year 2009 has been appended

Retrieving draft data for the year 2010
2010 draft data has been retrieved
Data for the year 2010 has been appended

Retrieving draft data for the year 2011
2011 draft data has been retrieved
Data for the year 2011 has been appended

Retrieving draft data for the year 2012
2012 draft data has been retrieved
Data for the year 2012 has been appended

Retrieving draft data for the year 2013
2013 draft data has been retrieved
Data for the year 2013 has be

In [20]:
# COLLECT NFL DATA

# Collect the NFL data
weekly_player_data = nfl.import_weekly_data(years= list(range(2004,2023)), downcast= True)

# Export the dataframe as a CSV file
weekly_player_data.to_csv(nfl_file_path)

Downcasting floats.
